In [1]:
from platform import python_version
print(python_version())
get_ipython().run_line_magic('pylab', 'inline')


3.9.7
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
#from tinc import *
import tinc
import os
import numpy as np
import djlib.clex as cl
import djlib as dj
from scipy.spatial import ConvexHull
import pickle
import matplotlib.pyplot as plt
from tinc import *

In [3]:
from IPython.core.display import display, HTML


/tmp/ipykernel_12074/3492823427.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [4]:
#Set up tinc stuff to let us view the ECI histogram plots later
eci_index = tinc.Parameter("eci_index", "index")
eci_index.values = np.array(list(range(253)))
graph_buffer = tinc.DiskBufferImage("graph", "out.png", "eci_hist_images")
ps = tinc.ParameterSpace("ps")
ps.register_parameters([eci_index])
ps.enable_cache()

Validating json with /home/muna/.local/conda/envs/py397/lib/python3.9/site-packages/tinc-0.9.53-py3.9.egg/tinc/tinc_cache_schema.json


In [5]:
datafile = '/mnt/DATA/Braid_Backup/munasaber/NiobiumOxides/TiNb2O7/Correct_Encut_CASM_LixTiNb2O7_LowestEnergy/HistogramMonteCarloCode/STAN/alpha_b_10_beta_b_16p66/STAN_executable.pkl'
with open(datafile, 'rb') as f:
    eci_set = pickle.load(f)
    eci_set = np.array(eci_set['eci'])
    eci_set = np.transpose(eci_set)
eci_set.shape

(500, 430)

In [6]:
#Code to make ECI histograms through tinc. 
def make_graph(eci_index):
    eci_hist_selection = eci_set[:,int(eci_index)]
    fig = cl.plot_eci_hist(eci_hist_selection)
    fig.set_size_inches(18, 10)
    #plt.axvline(x=lasso_eci[int(eci_index)], linewidth=5)
    fig = plt.gcf()
    image_path = os.path.join("/mnt/DATA/Braid_Backup/munasaber/NiobiumOxides/TiNb2O7/Correct_Encut_CASM_LixTiNb2O7_LowestEnergy/HistogramMonteCarloCode/STAN/alpha_b_10_beta_b_16p66/eci_hist_images/", 'eci_index_%d.png'%(eci_index))
    fig.savefig(image_path)
    file = open(image_path, 'rb')
    return file.read()


def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

eci_index.register_callback(value_changed)

In [7]:
#Tinc stuff to visualize the ECI histograms.
from ipywidgets import Button, HBox, VBox
box = VBox([eci_index.interactive_widget(), eci_index.interactive_control(), graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "80%"
box

Widget Javascript not detected.  It may not be installed or enabled properly.
